# Get the data

Depending on each case your data may look a little bit different but we should start our portfolio optimization by acquiring information on a set of STOCK to better understand their behaviour trough, in this case, one year of activity in the market.

In [9]:
import pandas as pd

data = pd.read_csv("binance_data6.csv")
data

,Unnamed: 0,Asset,Open time,Open,High,Low,Close,Volume,Closing time,Quote asset vol,Num traders,Taker buy base asset vol,Taker buy quote asset vol,To be ignored
0,0,ETHBTC,1634774400000,0.063057,0.066299,0.062904,0.065148,2.191795e+05,1634860799999,14185.381380,441192,1.002940e+05,6492.487566,0
1,1,ETHBTC,1634860800000,0.065160,0.065960,0.064685,0.065446,9.703463e+04,1634947199999,6338.327303,199872,5.274508e+04,3446.038753,0
2,2,ETHBTC,1634947200000,0.065445,0.068079,0.065181,0.067997,7.807181e+04,1635033599999,5183.565102,170954,3.906242e+04,2593.452923,0
3,3,ETHBTC,1635033600000,0.067998,0.068163,0.066101,0.067079,7.586534e+04,1635119999999,5087.916649,161735,3.649493e+04,2447.367339,0
4,4,ETHBTC,1635120000000,0.067081,0.067245,0.065269,0.066897,8.364134e+04,1635206399999,5546.270472,180063,4.119963e+04,2731.174031,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2191,361,LOOMBTC,1665964800000,0.000002,0.000003,0.000002,0.000003,4.744341e+07,1666051199999,129.065254,13690,2.295153e+07,62.933081,0
2192,362,LOOMBTC,1666051200000,0.000003,0.000003,0.000002,0.000003,2.686088e+07,1666137599999,69.917328,6809,1.220545e+07,31.913100,0
2193,363,LOOMBTC,1666137600000,0.000003,0.000003,0.000002,0.000002,6.054445e+06,1666223999999,15.126751,1519,2.269257e+06,5.706104,0
2194,364,LOOMBTC,1666224000000,0.000002,0.000003,0.000002,0.000002,3.657251e+06,1666310399999,8.986174,1068,1.792142e+06,4.413974,0


Following this we will calculate the average expected revenue for each asset. This is done by getting the difference between *opening* and *closing* time scaled to the opening price. This way we make sure each asset is independently evaluated.

We will also compute the covariance between each asset so that we can consider this values as part of our portfolio diversification constraint.

In [10]:
from cmath import exp
import numpy as np

# Unique asset list
asset_list = data["Asset"].unique()
#expected return
exp_ret = {}
return_list = []
for asset in asset_list:
    open_price = np.array(data[data["Asset"] == asset]["Open"].astype("float"))
    close_price = np.array(data[data["Asset"] == asset]["Close"].astype("float"))
        
    # Sign will be used to indicate the value gradient direction
    returns = ((close_price - open_price)/open_price)
    exp_ret[asset] = returns.mean()
    return_list.append(returns)

# Expected return on each asset
mu = [i for i in exp_ret.values()]   
    
# Compute covariance between returns
sigma = np.cov(np.vstack(return_list))

Here $\mu$ is the value associated with the expected average return for each asset.

In [11]:
for i,v in zip(asset_list, mu):
    print(f"Expected average return for asset {i} is {v}")

Expected average return for asset ETHBTC is 0.00043221237788004834
Expected average return for asset LTCBTC is -8.478051504195303e-05
Expected average return for asset BNBBTC is 0.0019369845670104667
Expected average return for asset NEOBTC is -0.0010735484309001167
Expected average return for asset QTUMETH is -0.0011878855362685353
Expected average return for asset LOOMBTC is 0.003263968784562174


And $\sigma$ is the covariance between those very same assets.

In [12]:
sigma

array([[ 0.00048188,  0.00022357,  0.00019375,  0.00021957, -0.00019389,
         0.00010709],
       [ 0.00022357,  0.00075058,  0.00022139,  0.00035433,  0.00022951,
         0.00021139],
       [ 0.00019375,  0.00022139,  0.00050048,  0.00023024,  0.00012029,
         0.00021223],
       [ 0.00021957,  0.00035433,  0.00023024,  0.00098326,  0.00046827,
         0.00045428],
       [-0.00019389,  0.00022951,  0.00012029,  0.00046827,  0.0010795 ,
         0.0003405 ],
       [ 0.00010709,  0.00021139,  0.00021223,  0.00045428,  0.0003405 ,
         0.00500931]])

It is important to know what the cost is of each asset so that we can also limit the budget we would like to spend in our investment.

In [13]:
filter = data.groupby("Asset").agg({"Open time":max}).reset_index()
costs = data.merge(filter, how='inner').drop_duplicates()
#print(costs)
cost_list = costs[["Asset","Open"]].to_dict('records')
print(cost_list)

[{'Asset': 'ETHBTC', 'Open': 0.067361}, {'Asset': 'LTCBTC', 'Open': 0.002689}, {'Asset': 'BNBBTC', 'Open': 0.014126}, {'Asset': 'NEOBTC', 'Open': 0.000407}, {'Asset': 'QTUMETH', 'Open': 0.002023}, {'Asset': 'LOOMBTC', 'Open': 2.45e-06}]


/var/folders/9k/xq3jd5c90w93s6tc_cws222r0000gn/T/ipykernel_96116/1294053305.py:1: FutureWarning: The provided callable <built-in function max> is currently using SeriesGroupBy.max. In a future version of pandas, the provided callable will be used directly. To keep current behavior pass the string "max" instead.
  filter = data.groupby("Asset").agg({"Open time":max}).reset_index()


We will store this information so that it can be used later.

In [14]:
import json

# Serializing json  
data = {"mu" : mu, "sigma": sigma.tolist(), "assets": cost_list} 
json_object = json.dumps(data, indent = 4)

with open("binance-data6.json", "w") as file:
    file.write(json_object)